## Library Imports

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix

import itertools
import os
import shutil
import random
from time import time
import matplotlib.pyplot as plt
#import pillow as pil
%matplotlib inline

In [2]:
DATA_PATH = 'SampleData'
#NUM_LAYERS_TO_REMOVE = 5
#NUM_LAYERS_TO_TRAIN = 20
LAYERS_TO_UNFREEZE = 0

## Create image pre-processing function

In [3]:
# Function to preprocess the image with Keras
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input
#def image_preprocessing(file):
#    img = image.load_img(file, target_size=(224, 224))
#    img_array = image.img_to_array(img)
#    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
#    return tf.keras.applications.mobilenet_v3.preprocess_input(img_array_expanded_dims)

## Load the metadata file.  Create a ground truth column.

In [4]:
# Load the metadata file
#metadata = pd.read_excel('metadata_modified.xlsx')
#image_path = metadata['Image Path']
# Create the ground truth values and put it into a new 'g_truth' column
#g_truth = metadata['Malignant'] + metadata['A']*2 + metadata['F']*4 + metadata['PT']*8 + metadata['TA']*16 + metadata['DC']*32 + metadata['LC']*64 + metadata['MC']*128 + metadata['PC']*256
#metadata['g_truth'] = g_truth

## Display a test image

In [6]:
mobile_v3 = tf.keras.applications.MobileNetV3Large()
#preprocessed_image = image_preprocessing(image_path[7117])
#predictions = mobile_v3.predict(preprocessed_image)
#print("Output shape: ", predictions.shape)
#results = imagenet_utils.decode_predictions(predictions)
#results

Remove the specifiied number of layers.
Freeze the specified number of layers.

## Define the training and validation datasets

In [8]:
path_to_files = 'BreaKHis_v1/BreaKHis_v1/histology_slides/breast/'
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  path_to_files,
  validation_split=0.25,
  subset="training",
  seed=42,
  image_size=(224, 224),
#  batch_size=32,
  shuffle=True
)
  
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  path_to_files,
  validation_split=0.25,
  subset="validation",
  seed=42,
  image_size=(224, 224),
#  batch_size=32,
  shuffle=True
)
#val_batches = tf.data.experimental.cardinality(val_dataset)
#val_dataset = val_dataset.take((2*val_batches) // 5)
#test_dataset = val_dataset.skip((2*val_batches) // 5)


Found 7909 files belonging to 8 classes.
Using 5932 files for training.
Found 7909 files belonging to 8 classes.
Using 1977 files for validation.


## Use AUTOTUNE to decrease I/O roadblocks

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
#test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

## Define the model

In [10]:
IMG_SIZE=(224,224)
IMG_SHAPE = IMG_SIZE + (3,)
def new_model (image_shape=IMG_SIZE):
    input_shape = image_shape + (3,)
    base_model = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                                        include_top=False, # important
                                                        weights='imagenet')
    
    # freeze the base model
    base_model.trainable = False

    # unfreeze some layers
    for layer in base_model.layers[-LAYERS_TO_UNFREEZE:]:
        layer.trainable = True

    print("Number of layers in the base model: ", len(base_model.layers))

    # create input layer
    inputs = tf.keras.Input(shape=input_shape)
    # pre-process inputs
    x = preprocess_input(inputs)
    # set training to False to avoid tracking statistics in batch norm layer
    x = base_model(x, training=False)
    # add flatten layer
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(8, activation='relu')(x)

    outputs = x

    model = tf.keras.Model(inputs,outputs)
    return model

In [11]:
# Define the learning rate scheduler
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow_addons as tfa

def lr_scheduler(lr, epoch_decay, layer_decay, model):
    optimizers_and_layers = []

    # Get all the trainable layers
    for layer in model.layers:
        if layer.name.startswith('Mobilenet'):
            for mobilenet_layer in layer.layers:
                if mobilenet_layer.trainable:
                    schedule = ExponentialDecay(initial_learning_rate=lr, decay_steps=1, decay_rate=epoch_decay)
                    optimizers_and_layers.append((tf.keras.optimizers.legacy.Adam(learning_rate=schedule), mobilenet_layer))
                    
        elif layer.trainable:
            schedule = ExponentialDecay(initial_learning_rate=lr, decay_steps=1, decay_rate=epoch_decay)
            optimizers_and_layers.append((tf.keras.optimizers.legacy.Adam(learning_rate=schedule), layer))

    # Create the learning rate schedule
    optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

    return optimizer

/Users/kdevoe/Documents/CS/Masters/AAI501/Group Project/aai501-su23-group-1/env7/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Train the model

In [14]:
def train_model(model, train_dataset, val_dataset, params):

    # Unpack parameters
    num_epochs = params['num_epochs']
    lr = params['lr']
    epoch_decay = params['epoch_decay']
    layer_decay = params['layer_decay']


    optimizer = lr_scheduler(lr, epoch_decay, layer_decay, model)

    model.compile(optimizer=optimizer,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    history = model.fit(
        train_dataset, 
        epochs=num_epochs,
        verbose=1,
        validation_data=val_dataset)

    return history

In [16]:
num_epochs = 2
lr = 0.001
epoch_decay = 0.5
layer_decay = 0.5

params = {
    'num_epochs': num_epochs,
    'lr': lr,
    'epoch_decay': epoch_decay,
    'layer_decay': layer_decay
}

model = new_model(IMG_SIZE)

history = train_model(model, train_dataset, val_dataset, params)

Number of layers in the base model:  263
Epoch 1/2
186/186 [==============================] - 79s 413ms/step - loss: 3.3211 - accuracy: 0.3296 - val_loss: 2.6529 - val_accuracy: 0.4117
Epoch 2/2
186/186 [==============================] - 75s 398ms/step - loss: 3.2900 - accuracy: 0.3299 - val_loss: 2.6529 - val_accuracy: 0.4117


In [19]:
print(history.history['accuracy'])

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Write the accuracy to a file



[0.32956844568252563, 0.32990559935569763]
